In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.5
SparkUI available at http://hail-m.us-central1-f.c.cncd-cncd.internal:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /hail-20210211-1045-0.2.61-3c86d3ba497a.log


In [ ]:
files = ["gs://pgr-wes-r20/GL_by_chrom/CNCD_Freeze_Two.2.GL.vcf.gz", "gs://pgr_wes_30k/GRCh38/chr2_30k_grch38.vcf.bgz",
        "gs://pgr-wgs-8k/VCF/vcf_DP10/pgr7k_chr2.vcf.gz", "gs://pgr-wgs-2k/chr2_PAK_indv.recode.vcf.gz"]

recode = {f"{i}":f"chr{i}" for i in (list(range(1, 23)) + ['X', 'Y'])}

pos = 'chr2:202123069-202123070'

for file in files:
    data = hl.import_vcf(file, reference_genome="GRCh38", force_bgz=True, contig_recoding=recode)
    ds_result1 = hl.filter_intervals(data, [hl.parse_locus_interval(pos, reference_genome='GRCh38')])
    ds_result1.rows().show()
    if ds_result1.count_rows() != 0:
        print('Annotate Heterozygous Alleles')
        ds_result1 = ds_result1.annotate_rows(hets = hl.agg.filter(ds_result1.GT.is_het(),hl.agg.collect(ds_result1.s)))
        ds_result1.count()
        print('Annotate N Hets')
        ds_result4 = ds_result1.annotate_rows(n_het = hl.agg.count_where(ds_result1.GT.is_het()))
        ds_result4.count()
        print('Annotate Homozygous Alleles')
        ds_result3 = ds_result4.annotate_rows(homs = hl.agg.filter(ds_result4.GT.is_hom_var(),hl.agg.collect(ds_result4.s)))
        ds_result3.count()
        print('Annotating N Homs')
        ds_result5 = ds_result3.annotate_rows(n_hom = hl.agg.count_where(ds_result3.GT.is_hom_var()))

        ds_result5.rows().export("gs://test1haseeb/Kashif-Lookup/"+file+pos+".tsv")
    

2021-02-11 10:46:26 Hail: INFO: Coerced sorted dataset
2021-02-11 10:46:26 Hail: INFO: reading 1 of 114 data partitions


+---------------+------------+------+---------+----------+----------------+
| locus         | alleles    | rsid |    qual | filters  | info.AF        |
+---------------+------------+------+---------+----------+----------------+
| locus<GRCh38> | array<str> | str  | float64 | set<str> | array<float64> |
+---------------+------------+------+---------+----------+----------------+
+---------------+------------+------+---------+----------+----------------+

+--------------+
| info.AQ      |
+--------------+
| array<int32> |
+--------------+
+--------------+

2021-02-11 10:47:02 Hail: INFO: Coerced sorted dataset
2021-02-11 10:47:02 Hail: INFO: reading 1 of 114 data partitions
2021-02-11 10:47:45 Hail: INFO: Coerced sorted dataset
2021-02-11 10:47:45 Hail: INFO: reading 1 of 202 data partitions


+---------------+------------+------+---------+----------+---------+----------+
| locus         | alleles    | rsid |    qual | filters  | info.AC | info.aaf |
+---------------+------------+------+---------+----------+---------+----------+
| locus<GRCh38> | array<str> | str  | float64 | set<str> |   int32 |  float64 |
+---------------+------------+------+---------+----------+---------+----------+
+---------------+------------+------+---------+----------+---------+----------+

+---------------+------------+---------------+------------+------------+
| info.callRate | info.hetRA | info.hg19_uid | info.homAA | info.homRR |
+---------------+------------+---------------+------------+------------+
|       float64 |      int32 | str           |      int32 |      int32 |
+---------------+------------+---------------+------------+------------+
+---------------+------------+---------------+------------+------------+

+---------------+---------------+---------------+---------------+
| info.maxDepth | info.maxHetAB | info.maxHomAB | info.medDepth |
+---------------+---------------+---------------+---------------+
|         int32 |       float64 |       float64 |         int32 |
+---------------+---------------+---------------+---------------+
+---------------+---------------+---------------+---------------+

+---------------+---------------+---------------+---------------+
| info.medHetAB | info.medHomAB | info.minDpeth | info.minHetAB |
+---------------+---------------+---------------+---------------+
|       float64 |       float64 |         int32 |       float64 |
+---------------+---------------+---------------+---------------+
+---------------+---------------+---------------+---------------+

+---------------+-------------+----------------------+--------------+
| info.minHomAB | info.noCall | info.oldMultiAllelic | info.pvalHwe |
+---------------+-------------+----------------------+--------------+
|       float64 |       int32 | str                  |      float64 |
+---------------+-------------+----------------------+--------------+
+---------------+-------------+----------------------+--------------+

2021-02-11 10:48:26 Hail: INFO: Coerced sorted dataset
2021-02-11 10:48:26 Hail: INFO: reading 1 of 202 data partitions
2021-02-11 10:52:37 Hail: INFO: Coerced sorted dataset
2021-02-11 10:52:37 Hail: INFO: reading 1 of 306 data partitions


...

2021-02-11 11:01:34 Hail: INFO: Coerced sorted dataset
2021-02-11 11:01:35 Hail: INFO: reading 1 of 306 data partitions


Annotate Homozygous Alleles


2021-02-11 11:03:29 Hail: INFO: Coerced sorted dataset
2021-02-11 11:03:29 Hail: INFO: reading 1 of 306 data partitions


Annotating N Homs


2021-02-11 11:05:23 Hail: INFO: Coerced sorted dataset
2021-02-11 11:05:23 Hail: INFO: reading 1 of 306 data partitions
2021-02-11 11:10:13 Hail: INFO: merging 1 files totalling 1.2K...
2021-02-11 11:10:13 Hail: INFO: while writing:
    gs://test1haseeb/Kashif-Lookup/gs://pgr-wgs-8k/VCF/vcf_DP10/pgr7k_chr2.vcf.gzchr2:202123069-202123070.tsv
  merge time: 487.671ms
2021-02-11 11:12:26 Hail: INFO: Coerced sorted dataset
2021-02-11 11:12:26 Hail: INFO: reading 1 of 1035 data partitions


In [ ]:
ds_result1.count()

In [ ]:
# if ds_result1.count()[0] != 0:
ds_result1.count_rows()

In [ ]:
data6 = hl.read_matrix_table("gs://test1haseeb/GOLGA6L6_20K_vep.mt")
data6.describe()

In [ ]:
data6 = hl.read_matrix_table("gs://test1haseeb/GOLGA6L6_20K_vep.mt")
data1 = data6.annotate_rows(
    info=data6.info.annotate(GeneName=data6.vep.transcript_consequences.gene_symbol,
                             EnsembleGeneId=data6.vep.transcript_consequences.gene_id,
                             variantConsequence=data6.vep.most_severe_consequence,
                             consequence_terms=data6.vep.transcript_consequences.consequence_terms,
                             transcript_affected=data6.vep.transcript_consequences.transcript_id,
                             lof=data6.vep.transcript_consequences.lof,
                             lof_filter=data6.vep.transcript_consequences.lof_filter,
                             biotype=data6.vep.transcript_consequences.biotype,
                             canonical=data6.vep.transcript_consequences.canonical
                             # should we add csq_score????
                               ))
data1.rows().show()
data2 = data1.rows().to_pandas()
data2.to_excel('GOLGA6L6_20K_vep.xlsx')
!gsutil cp GOLGA6L6_20K_vep.xlsx gs://test1haseeb/var-lookup/

In [ ]:
data6 = hl.read_matrix_table("gs://test1haseeb/GALNT9_30K_vep.mt")
data1 = data6.annotate_rows(
    info=data6.info.annotate(GeneName=data6.vep.transcript_consequences.gene_symbol,
                             EnsembleGeneId=data6.vep.transcript_consequences.gene_id,
                             variantConsequence=data6.vep.most_severe_consequence,
                             consequence_terms=data6.vep.transcript_consequences.consequence_terms,
                             transcript_affected=data6.vep.transcript_consequences.transcript_id,
                             lof=data6.vep.transcript_consequences.lof,
                             lof_filter=data6.vep.transcript_consequences.lof_filter,
                             biotype=data6.vep.transcript_consequences.biotype,
                             canonical=data6.vep.transcript_consequences.canonical
                             # should we add csq_score????
                               ))
data1.rows().show()
data2 = data1.rows().to_pandas()
data2.to_excel('GALNT9_30K_vep.xlsx')
!gsutil cp GALNT9_30K_vep.xlsx gs://test1haseeb/var-lookup/
# data1.rows().export("gs://test1haseeb/var-lookup/GALNT9_30K_vep.tsv")

In [ ]:
data6 = hl.read_matrix_table("gs://test1haseeb/KIR3DL1_20K_vep.mt")
data1 = data6.annotate_rows(
    info=data6.info.annotate(GeneName=data6.vep.transcript_consequences.gene_symbol,
                             EnsembleGeneId=data6.vep.transcript_consequences.gene_id,
                             variantConsequence=data6.vep.most_severe_consequence,
                             consequence_terms=data6.vep.transcript_consequences.consequence_terms,
                             transcript_affected=data6.vep.transcript_consequences.transcript_id,
                             lof=data6.vep.transcript_consequences.lof,
                             lof_filter=data6.vep.transcript_consequences.lof_filter,
                             biotype=data6.vep.transcript_consequences.biotype,
                             canonical=data6.vep.transcript_consequences.canonical
                             # should we add csq_score????
                               ))
data1.rows().show()
data2 = data1.rows().to_pandas()
data2.to_excel('KIR3DL1_20K_vep.xlsx')
!gsutil cp KIR3DL1_20K_vep.xlsx gs://test1haseeb/var-lookup/
# data1.rows().export("gs://test1haseeb/var-lookup/KIR3DL1_20K_vep.tsv")

In [ ]:
data6 = hl.read_matrix_table("gs://test1haseeb/LYPLAL1_20K.mt")
data1 = data6.annotate_rows(
    info=data6.info.annotate(GeneName=data6.vep.transcript_consequences.gene_symbol,
                             EnsembleGeneId=data6.vep.transcript_consequences.gene_id,
                             variantConsequence=data6.vep.most_severe_consequence,
                             consequence_terms=data6.vep.transcript_consequences.consequence_terms,
                             transcript_affected=data6.vep.transcript_consequences.transcript_id,
                             lof=data6.vep.transcript_consequences.lof,
                             lof_filter=data6.vep.transcript_consequences.lof_filter,
                             biotype=data6.vep.transcript_consequences.biotype,
                             canonical=data6.vep.transcript_consequences.canonical
                             # should we add csq_score????
                               ))
data1.rows().show()
data2 = data1.rows().to_pandas()
data2.to_excel('LYPLAL1_20K_vep.xlsx')
!gsutil cp LYPLAL1_20K_vep.xlsx gs://test1haseeb/var-lookup/
# data1.rows().export("gs://test1haseeb/var-lookup/LYPLAL1_20K_vep.tsv")

In [ ]:
data6 = hl.read_matrix_table("gs://test1haseeb/LYPLAL1_30K_vep.mt")
data1 = data6.annotate_rows(
    info=data6.info.annotate(GeneName=data6.vep.transcript_consequences.gene_symbol,
                             EnsembleGeneId=data6.vep.transcript_consequences.gene_id,
                             variantConsequence=data6.vep.most_severe_consequence,
                             consequence_terms=data6.vep.transcript_consequences.consequence_terms,
                             transcript_affected=data6.vep.transcript_consequences.transcript_id,
                             lof=data6.vep.transcript_consequences.lof,
                             lof_filter=data6.vep.transcript_consequences.lof_filter,
                             biotype=data6.vep.transcript_consequences.biotype,
                             canonical=data6.vep.transcript_consequences.canonical
                             # should we add csq_score????
                               ))
data1.rows().show()
data2 = data1.rows().to_pandas()
data2.to_excel('LYPLAL1_30K_vep.xlsx')
!gsutil cp LYPLAL1_30K_vep.xlsx gs://test1haseeb/var-lookup/
# data1.rows().export("gs://test1haseeb/var-lookup/LYPLAL1_30K_vep.tsv")